# Extracting Text

In [3]:
import fitz  # PyMuPDF is imported as fitz

In [4]:
# Open the PDF document
def extract_text_from_pdf(pdf_file_path):
    test_doc = fitz.open(pdf_file_path)  # change the path to the pdf file

    # Define thresholds to extract headers and footers
    header_threshold = 0.15  # Top 15% of the page height
    footer_threshold = 0.925  # Bottom 7.5% of the page height

    ## EXTRACT THE TEXT BY BLOCKS

    # Initialize a variable to hold the extracted text
    extracted_text_block = []
    text_string = ""

    # Loop through each page
    for page_num in range(len(test_doc)):
        page = test_doc.load_page(page_num)  # Load the page
        text_block = page.get_text("blocks")  # Extract text from the page as blocks
        page_height = page.rect.height  # Get the height of the page

        for block in text_block:
            # Get the vertical position of the block (block[1] is the top y-coordinate, block[3] is the bottom y-coordinate)
            block_top = block[1]
            block_bottom = block[3]

            # Filter out blocks that are within the header or footer region
            if (block_top > page_height * header_threshold) and (
                block_bottom < page_height * footer_threshold
            ):
                text = block[
                    4
                ]  # The text content is in the 5th element of the block tuple
                text_string += text + "\n\n"  # Write the block text
    # Close the document
    test_doc.close()
    return text_string

In [6]:
original_minute = r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\cv-tech-llm\DATA\documents\enero\Acta Sesión Ordinaria No. 045 2024-01-02.pdf"

extracted_minute = extract_text_from_pdf(original_minute)
print(extracted_minute)

SESIÓN DE CONCEJO No. 045 DE 02 DE ENERO DE 2024 


 


SUMARIO: 


CAPÍTULOS   
 
 
TEMA 


 


I. 
Verificación del cuórum. 


 


II. 
Instalación de la sesión. 


 


III. 
Lectura del Orden del Día.   


 


IV. 
Himno a Quito.  


 


V. 
Primer debate del Proyecto de Ordenanza Metropolitana que 
Regula y Limita el uso de Pirotecnia en el Distrito Metropolitano 
de Quito (IC-001-CSA-2022).  


 
 


VI. 
Primer debate del Proyecto de "Ordenanza Metropolitana 
Reformatoria del Código Municipal para el Distrito Metropolitano 
de Quito, por la cual se incorpora las Normas para Regular el Uso 
del Subsistema de Transporte Público Metro de Quito" (IC-ORD-
CM-2023-002). 


 
 


VII. 
Clausura de la sesión. 


 


Anexo. 


 


 


 


 


 


 


 


 


 


SESIÓN DE CONCEJO No. 045 DE 02 DE ENERO DE 2024 


 


INDICE: 


CAPÍTULO  
 
 
    TEMA 
 
 
 
         PÁGINA 


I. 
Verificación del cuórum.  .................................................................................  

# Connection to OpenAI API

In [7]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [8]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

##CHANGE EVERYTIME THE PATH
extracted_raw_data = extracted_minute
text_content = []

# Count the number of tokens    
text_tokens = encoding.encode(extracted_raw_data)
num_tokens = len(text_tokens)
print(f"Number of tokens : {num_tokens}")

text_content.append(extracted_raw_data) #save for future processing
     
print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 28285

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [6]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [9]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]
client = OpenAI(api_key = key_ct)

### Prompt Engineering (Zero-shot)

### First approach

In [ ]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
full_instruction = """Has un resumen de una minuta de reunión de un gobierno local con las siguientes instrucciones:
-  Enfatiza los principales temas que se hablaron en la sesion, problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan cuando sea necesario.
    - Provee datos concretos y no des detalles irrelevantes
    - No emitas juicios de valor
    - Integra las secciones por tema si estos se parecen entre sí
  -   Envía la información con la siguiente estructura
      {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesion (Ordinaria, Extraordinaria o Conmemorativa) No. (número de la sesion)", 
     "content:  [" resumen de la sesion en donde se explique de manera fluida y clara con las instrucciones enviadas arriba con este formato 'Titulo 1: Resumen del tema 1','Titulo 2: Resumen del tema 2'...""], 
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


### Second Approach

In [16]:
# token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# Instruction text
full_instruction = """Eres un asistente que resume actas de sesiones del Concejo Metropolitano de Quito con el objetivo de entregar informacion relevante a los ciudadanos.
      - Identifica los principales temas discutidos en la sesión
      -  Enfatiza  problemas, peticiones y soluciones o acuerdos que se llegan.
      - Identifica datos cuantitativos concretos si se tratan en la sesion 
      - Indica los nombres y los cargos de las personas  que participan cuando sea necesario.
      - Sé imparcial
  -   Envía la información con la siguiente estructura
      {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesion (Ordinaria, Extraordinaria o Conmemorativa) No. (número de la sesion)", 
     "content:  [" resumen de la sesion" ],	
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 197


### CoT approach

In [ ]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
full_instruction = """Eres un asistente que resume minutas de reunión de un gobierno local con el objetivo de dar informacion relevante a la ciudadania
Razona a través de los siguientes pasos:
Paso 1: Identifica los temas principales tratados en la sesión. 
Paso 2: Integra en una sola oración todo lo relacionado a la instalación de la sesión, cuórum y lectura del orden del dia.
Paso 3: Identifica problemas, peticiones y soluciones o acuerdos que se llegan, indicando los nombres de las personas que participan.
Paso 4: Resume cada tema en un parrafo de 3-5 oraciones.  
Paso 5: Unifica los temas que tengan relación en una sección. 
Paso 6: Coloca titulos sin numeracion en cada sección de acuerdo a los temas tratados. 
Paso 7: Escribe el resumen general de la sesión con fluencia y claridad. 
Paso 8: Envía la información con la siguiente estructura {
     "date" : "yyyy-mm-dd", 
     "document_id" : "acta_0xx(numero_del_acta)", 
     "act_name": "Sesión (Ordinaria, Extraordinaria o Commemorativa) No. (número de la sesion)", 
     "content":  ["resumen de la sesion con cada seccion con su titulo como un item diferente: 'Titulo 1: Resumen del tema 1','Titulo 2: Resumen del tema 2'..."], 
    } 
    """
# Encode the text
instruction_tokens = encoding.encode(full_instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")

### cleanse instruction

In [13]:
full_instruction = "Has un resumen del siguiente documento:"

### Sending instructions

In [17]:

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : full_instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.80,
    max_tokens = 2000,
)


In [18]:

response_summary_vers001 = response.choices[0].message.content
print(response_summary_vers001)   

{
  "date": "2024-01-02", 
  "document_id": "acta_045", 
  "act_name": "Sesion Ordinaria No. 045", 
  "content": [
    "En la sesión ordinaria del Concejo Metropolitano de Quito del 2 de enero de 2024, se discutieron dos proyectos de ordenanza en primer debate: uno que regula y limita el uso de pirotecnia en el Distrito Metropolitano de Quito y otro que reforma el Código Municipal para regular el uso del servicio del Metro de Quito. ",
    "El concejal Andrés Campaña, autor del primer proyecto, destacó la necesidad de regular la pirotecnia debido a sus efectos perjudiciales sobre la salud pública, la fauna y el medio ambiente. Se mencionaron datos sobre el incremento de accidentes y problemas de salud relacionados con el uso de pirotecnia. Se propuso la participación de diversos sectores en el desarrollo de un informe de segundo debate. El Comandante del Cuerpo de Bomberos, Esteban Cárdenas, también participó, sugiriendo que el proyecto regrese a la Comisión para revisar las recomendac

### Opcional JSON formatting

In [ ]:

# #extract ther markdown labels

import re

def extract_markdown_content(text):
    # Regular expression to find content between triple backticks
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

# Example usage

response_summary_vers001 = extract_markdown_content(response_summary_vers001)
print(response_summary_vers001)


# MongoDB Connection

In [19]:
import json
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv(override=True)
db_url = os.environ["DATABASE_URL"]

In [20]:
client = MongoClient(db_url)
db = client["cv_tech"]
collection = db["Summary"]
client.admin.command('ping')

{'ok': 1}

In [21]:
response_summary_dict = json.loads(response_summary_vers001)

In [22]:

# Convert the content to a dictionary
try:
    response_summary_dict = json.loads(response_summary_vers001)
    # Add a new key-value pair to the dictionary - CHANGE EVERYTIME
    response_summary_dict["document_url"] = "https://www7.quito.gob.ec/mdmq_ordenanzas/Administraci%C3%B3n%202023-2027/Actas%20Sesiones/2024/Acta%20Sesio%CC%81n%20Ordinaria%20No.%20045%202024-01-02.pdf"

    print(response_summary_dict)
    
    
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

{'date': '2024-01-02', 'document_id': 'acta_045', 'act_name': 'Sesion Ordinaria No. 045', 'content': ['En la sesión ordinaria del Concejo Metropolitano de Quito del 2 de enero de 2024, se discutieron dos proyectos de ordenanza en primer debate: uno que regula y limita el uso de pirotecnia en el Distrito Metropolitano de Quito y otro que reforma el Código Municipal para regular el uso del servicio del Metro de Quito. ', 'El concejal Andrés Campaña, autor del primer proyecto, destacó la necesidad de regular la pirotecnia debido a sus efectos perjudiciales sobre la salud pública, la fauna y el medio ambiente. Se mencionaron datos sobre el incremento de accidentes y problemas de salud relacionados con el uso de pirotecnia. Se propuso la participación de diversos sectores en el desarrollo de un informe de segundo debate. El Comandante del Cuerpo de Bomberos, Esteban Cárdenas, también participó, sugiriendo que el proyecto regrese a la Comisión para revisar las recomendaciones técnicas que no

### Insert to MongoDB collection

In [23]:
try:
    result = collection.insert_one(response_summary_dict)
    print(f"Inserted document ID: {result.inserted_id}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Inserted document ID: 67212e88fece4612de742347


### Save copy to JSON data file

In [ ]:

# Write the JSON data to a new file
#CHANGE THIS FILE NAME
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\data\json_files\c_o_t_example", "w", encoding="utf-8") as json_file: #CHANGE THIS FILE NAME
    json.dump(response_summary_dict, json_file, ensure_ascii=False, indent=4)

print("Text extraction completed and saved as JSON.")